In [1]:
import pandas as pd
file_path = r'/kaggle/input/wallets_features_classes_combined.csv'
df_wallets_features_classes_combined = pd.read_csv(file_path)

In [2]:
import optuna

In [3]:
# 2. Preprocess data
df_wallets_classification = df_wallets_features_classes_combined.drop(columns=['Time step']).drop_duplicates()
data = df_wallets_classification.loc[(df_wallets_classification['class'] != 3), 'address']
df_wallets_feature_selected = df_wallets_classification.loc[df_wallets_classification['address'].isin(data)]

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import VotingClassifier
from sklearn.base import clone 
import xgboost as xgb
from sklearn.base import clone
import numpy as np

In [7]:
# Define X and y
y = df_wallets_feature_selected[['class']]
y = y['class'].apply(lambda x: 0 if x == 2 else 1 )
X = df_wallets_feature_selected.drop(columns=['address', 'class'])

# Train/Test Split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=False)

In [9]:
X_train.sample(3)

,num_txs_as_sender,num_txs_as receiver,first_block_appeared_in,last_block_appeared_in,lifetime_in_blocks,total_txs,first_sent_block,first_received_block,num_timesteps_appeared_in,btc_transacted_total,...,blocks_btwn_output_txs_min,blocks_btwn_output_txs_max,blocks_btwn_output_txs_mean,blocks_btwn_output_txs_median,num_addr_transacted_multiple,transacted_w_address_total,transacted_w_address_min,transacted_w_address_max,transacted_w_address_mean,transacted_w_address_median
423704,0.0,1.0,487969.0,487969.0,0.0,1.0,0.0,487969.0,1.0,4.890607,...,0.0,0.0,0.0,0.0,0.0,101.0,1.0,1.0,1.0,1.0
169032,1.0,0.0,479905.0,479905.0,0.0,1.0,479905.0,0.0,1.0,0.004487,...,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0,1.0
605268,1.0,0.0,405322.0,405322.0,0.0,1.0,405322.0,0.0,1.0,0.086663,...,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0,1.0


In [10]:
# Scaling
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

#Flatten into 1 dimension array cuz scikit-learn & f1-score needs
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

In [11]:
X_test_scaled.sample(3)

,num_txs_as_sender,num_txs_as receiver,first_block_appeared_in,last_block_appeared_in,lifetime_in_blocks,total_txs,first_sent_block,first_received_block,num_timesteps_appeared_in,btc_transacted_total,...,blocks_btwn_output_txs_min,blocks_btwn_output_txs_max,blocks_btwn_output_txs_mean,blocks_btwn_output_txs_median,num_addr_transacted_multiple,transacted_w_address_total,transacted_w_address_min,transacted_w_address_max,transacted_w_address_mean,transacted_w_address_median
12933,0.0,0.001862,0.666629,0.666629,0.0,0.0,0.0,0.933886,0.0,4.536937e-07,...,0.0,0.0,0.0,0.0,0.0,0.000740,0.0,0.0,0.0,0.0
36542,0.0,0.001862,0.020832,0.020832,0.0,0.0,0.0,0.805812,0.0,2.809152e-08,...,0.0,0.0,0.0,0.0,0.0,0.000026,0.0,0.0,0.0,0.0
1548,0.0,0.001862,0.979127,0.979127,0.0,0.0,0.0,0.995860,0.0,3.122339e-07,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [12]:
y_train

array([0, 0, 0, ..., 0, 0, 0])

In [13]:
y_test

array([0, 1, 0, ..., 0, 0, 0])

## Retrain model

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (
    precision_recall_fscore_support,
    f1_score,
    accuracy_score,
    classification_report,
    roc_auc_score
)

In [15]:
def evaluate_model(model_name, y_true, y_pred, y_proba=None):
    # Basic metrics for class 1 (positive class)
    prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_pred, zero_division=0)
    micro_f1 = f1_score(y_true, y_pred, average='micro')
    accuracy = accuracy_score(y_true, y_pred)
    
    # ROC AUC - Good for class imbalance.
    roc_auc = None
    if y_proba is not None and len(set(y_true)) == 2:
        roc_auc = roc_auc_score(y_true, y_proba)

    # Print section
    print(f"\n {model_name} Evaluation Results:")
    
    if roc_auc is not None:
        print("ROC AUC: %.6f" % roc_auc)
        
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, digits=3))

In [16]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_scaled, y_train)
y_pred_rf = rf.predict(X_test_scaled)
evaluate_model("Random Forest", y_test, y_pred_rf)


 Random Forest Evaluation Results:

Classification Report:
              precision    recall  f1-score   support

           0      0.992     0.997     0.995     76718
           1      0.921     0.790     0.850      2889

    accuracy                          0.990     79607
   macro avg      0.956     0.893     0.922     79607
weighted avg      0.990     0.990     0.990     79607



In [17]:
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb.fit(X_train_scaled, y_train)
y_pred_xgb = xgb.predict(X_test_scaled)
evaluate_model("XGBoost", y_test, y_pred_xgb)


 XGBoost Evaluation Results:

Classification Report:
              precision    recall  f1-score   support

           0      0.993     0.996     0.994     76718
           1      0.888     0.809     0.846      2889

    accuracy                          0.989     79607
   macro avg      0.940     0.902     0.920     79607
weighted avg      0.989     0.989     0.989     79607



# Use best model for feature selection

## PFI (Permutation feature importance)

In [18]:
import eli5
from eli5.sklearn import PermutationImportance as PI
from sklearn.base import clone
# PFI Calculations
perm_rf = PI(rf, random_state=42).fit(X_test_scaled, y_test)
pfi_rf = eli5.explain_weights_df(perm_rf, feature_names=X_test_scaled.columns.tolist())
pfi_rf = pfi_rf[['feature', 'weight']].rename(columns={'weight': 'weight_rf'})

2025-06-04 09:33:32.372798: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749029612.693781      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749029612.781859      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## DCFI (Drop Column Feature Importance)

In [ ]:
# X_train_full, y_train_full is the full training dataset (Scaled)
# X_eval, y_eval is the full testing dataset (Scaled)
def compute_dcfi_rf_only(model_template, X_train_full, y_train_full, X_eval, y_eval, random_state=42):
    
    y_train_array = y_train_full.ravel()
    y_eval_array = y_eval.ravel()

    #Clone the root model 
    model_clone_benchmark = clone(model_template)
    if hasattr(model_clone_benchmark, 'random_state'):
        model_clone_benchmark.random_state = random_state

    # Convert to numpy array if needed
    X_train_full_array = X_train_full.values if isinstance(X_train_full, pd.DataFrame) else X_train_full
    X_eval_array = X_eval.values if isinstance(X_eval, pd.DataFrame) else X_eval

    # Train the model using all features from training data
    model_clone_benchmark.fit(X_train_full_array, y_train_array)
    y_pred_benchmark = model_clone_benchmark.predict(X_eval_array)
    benchmark_score = f1_score(y_eval_array, y_pred_benchmark, pos_label=1, average='binary')

    importances = []
    features = X_train_full.columns.tolist() if isinstance(X_train_full, pd.DataFrame) else [f'Feature_{i}' for i in range(X_train_full_array.shape[1])]

    for i in range(X_train_full_array.shape[1]):
        # Assign exactly the random_state initialized be4
        model_clone_drop = clone(model_template)
        if hasattr(model_clone_drop, 'random_state'):
            model_clone_drop.random_state = random_state

        X_train_subset_array = np.delete(X_train_full_array, i, axis=1)
        X_eval_subset_array = np.delete(X_eval_array, i, axis=1)
        
        model_clone_drop.fit(X_train_subset_array, y_train_array)
        y_pred_drop = model_clone_drop.predict(X_eval_subset_array)
        drop_col_score = f1_score(y_eval_array, y_pred_drop, pos_label=1, average='binary', zero_division=0)
        importances.append(benchmark_score - drop_col_score)

    dcfi_df_temp = pd.DataFrame({
        'Feature': features,
        'Imp': importances
    })
    return dcfi_df_temp.sort_values('Imp', ascending=False)

print("\nCalculating DCFI for RF model (evaluated on X_test_scaled)...")
dcfi_rf_df = compute_dcfi_rf_only(rf, X_train_scaled, y_train, X_test_scaled, y_test)
dcfi_rf_df = dcfi_rf_df.rename(columns={'Imp': 'DCFI_Score'})

print("Top 5 DCFI features for RF:")
print(dcfi_rf_df.sort_values(by='DCFI_Score', ascending=False).head())


Calculating DCFI for RF model (evaluated on X_test_scaled)...


In [ ]:
# Cell 17 - Visualizing PFI and DCFI Separately (Adjusted for your column names)
import matplotlib.pyplot as plt
import seaborn as sns

num_features_to_plot = 20

# --- Individual PFI Scores ---
# pfi_rf_df has 'Feature' and 'weight_rf'
top_pfi_features_df = pfi_rf_df.sort_values(by='weight_rf', ascending=False).head(num_features_to_plot)

# --- Individual DCFI Scores ---
# dcfi_rf_df has 'Feature' and 'DCFI_Score'
top_dcfi_features_df = dcfi_rf_df.sort_values(by='DCFI_Score', ascending=False).head(num_features_to_plot)

# Create subplots
fig, axes = plt.subplots(2, 1, figsize=(15, 15))

# Plot Top PFI Features
sns.barplot(x='weight_rf', y='Feature', data=top_pfi_features_df, ax=axes[0], palette="viridis", orient='h')
axes[0].set_title(f'Top {num_features_to_plot} Features by PFI (RF Model - Score: weight_rf)')
axes[0].set_xlabel("PFI Importance Score (weight_rf)")
axes[0].set_ylabel("Feature")

# Plot Top DCFI Features
sns.barplot(x='DCFI_Score', y='Feature', data=top_dcfi_features_df, ax=axes[1], palette="magma", orient='h')
axes[1].set_title(f'Top {num_features_to_plot} Features by DCFI (RF Model - Score: DCFI_Score)')
axes[1].set_xlabel("DCFI Importance Score")
axes[1].set_ylabel("Feature")

plt.tight_layout()
plt.show()

In [ ]:
# Cell 16 - Feature Selection by Union 
num_top_features_each_method = 20

top_pfi_list = pfi_rf_df.sort_values(by='weight_rf', ascending=False).head(num_top_features_each_method)['Feature'].tolist()
top_dcfi_list = dcfi_rf_df.sort_values(by='DCFI_Score', ascending=False).head(num_top_features_each_method)['Feature'].tolist()

selected_features_union = list(set(top_pfi_list + top_dcfi_list)) # Get unique features from both lists

print(f"\nNumber of features in top {num_top_features_each_method} PFI list: {len(top_pfi_list)}")
print(f"Features from PFI top {num_top_features_each_method}: {top_pfi_list}") 

print(f"Number of features in top {num_top_features_each_method} DCFI list: {len(top_dcfi_list)}")
print(f"Features from DCFI top {num_top_features_each_method}: {top_dcfi_list}")

print(f"Number of unique features from the union of top {num_top_features_each_method} PFI and DCFI: {len(selected_features_union)}")
print(f"Selected features (union): {selected_features_union}")

In [ ]:
# Uses 'selected_features_union' from the previous cell 

X_train_selected_rf_union = X_train_scaled[selected_features_union].copy()
X_test_selected_rf_union = X_test_scaled[selected_features_union].copy()

print(f"\nShape of X_train_selected_rf_union: {X_train_selected_rf_union.shape}")
print(f"Shape of X_test_selected_rf_union: {X_test_selected_rf_union.shape}")